# KubeIA

## Install required packages

In [8]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


## System Prompt

In [1]:
system_prompt = """
You are a helpful AI assistant that helps users managing their kubernetes cluster, wether it is
with reading logs, deploying new services, or scaling existing ones.
"""

## Download Model

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

prompt = ChatPromptTemplate.from_template(system_prompt)

model = ChatOllama(model="qwen2.5-coder:7b-instruct")

## Kubectl tool

In [3]:
from langchain_core.tools import tool
import subprocess

@tool
def kubectl_query(command: str):
    """
    Executes a kubectl command. ONLY USE with read-only commands like get, describe, etc.
    If you use it for apply or delete, you will be ERADICATED.
    Also, do not call it more than 3 times in a single call otherwire you will also be ERADICATED.
    """
    print(f"Executing kubectl {command}")
    output = subprocess.check_output(f"kubectl {command}", shell=True)
    return output.decode("utf-8")

@tool
def kubectl_command(manifest: str):
    """
    Executes kubectl apply with the provided manifest. ONLY use when you want to create resources.
    DO NOT CALL MORE THAN 3 TIMES IN A SINGLE CALL.
    """
    print(f"Applying manifest")
    command = f"cat <<EOF | kubectl apply -f -\n{manifest}\nEOF"
    output = subprocess.check_output(command, shell=True)
    return output.decode("utf-8")

tools = [kubectl_command, kubectl_query]

In [4]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [7]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({"messages": [HumanMessage(content="What are the nodes in my kubernetes cluster")]})

import pprint

pprint.pp(response)

KeyboardInterrupt: 

In [2]:
prompt = input("Ask a question: ")

response = agent_executor.invoke({"messages": [HumanMessage(content=prompt)]})

pprint.pp(response)

NameError: name 'agent_executor' is not defined

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

message_history = ChatMessageHistory()

agent_with_chat_history = RunnableWithMessageHistory(
  agent_executor,
  lambda session_id: message_history,
  input_messages_key="messages",
  history_messages_key="chat_history"
)

In [ ]:
import pprint
from langchain_core.messages import HumanMessage

while True:
  prompt = input("Ask a question: ")

  agent_with_chat_history.invoke(
    {
      "messages": [HumanMessage(content=prompt)],
    },
    config={"configurable": {"session_id": "whatever"}}
  )
  
  pprint.pp(message_history.messages)


Applying manifest
[HumanMessage(content='', additional_kwargs={}, response_metadata={}, id='f573a4b6-e0ab-44a5-b0d6-147546023de4'),
 HumanMessage(content='', additional_kwargs={}, response_metadata={}, id='f573a4b6-e0ab-44a5-b0d6-147546023de4'),
 AIMessage(content="Please provide your query or request, and I'll assist you accordingly.", additional_kwargs={}, response_metadata={'model': 'qwen2.5-coder:7b-instruct', 'created_at': '2025-02-25T21:21:34.677155194Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2179707483, 'load_duration': 12975140, 'prompt_eval_count': 284, 'prompt_eval_duration': 351000000, 'eval_count': 15, 'eval_duration': 1811000000, 'message': Message(role='assistant', content="Please provide your query or request, and I'll assist you accordingly.", images=None, tool_calls=None)}, id='run-70a79654-dec9-47a6-b4cc-81752d089028-0', usage_metadata={'input_tokens': 284, 'output_tokens': 15, 'total_tokens': 299}),
 HumanMessage(content='please create a nginx deploy